In [1]:
%pip install -q keras-tuner

You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from tensorflow.keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.astype('float32') / 255.
test_images = test_images.astype('float32') / 255.

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from keras_tuner import Hyperband, HyperParameters

import pathlib
import shutil
import tempfile

def model_builder(hp: HyperParameters):
    hp_units = hp.Int('level-2-units', min_value=32, max_value=512, step=32)
    model = Sequential([
        Flatten(input_shape=(28, 28)),
        Dense(hp_units, activation='relu'),
        Dense(10)
    ])

    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    model.compile(
        optimizer=Adam(learning_rate=hp_learning_rate),
        loss=SparseCategoricalCrossentropy(from_logits=True),
        metrics=['accuracy'])

    return model

tuner_root = pathlib.Path(tempfile.mkdtemp() + 'tuner')
shutil.rmtree(tuner_root, ignore_errors=True)
tuner = Hyperband(
    model_builder, objective='val_accuracy', max_epochs=10, factor=3, 
    directory=str(tuner_root), project_name='intro')

In [12]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=5)
tuner.search(train_images, train_labels, epochs=50, validation_split=0.2, callbacks=[early_stopping])

best_hyperparams = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"""
    best hyperparams: 
    level 2 unit {best_hyperparams.get("level-2-units")} 
    learning rate {best_hyperparams.get("learning_rate")}
    """)

INFO:tensorflow:Oracle triggered exit

    best hyperparams: 
    level 2 unit 416 
    learning rate 0.001
    


In [13]:
model: Sequential = tuner.hypermodel.build(best_hyperparams)

model.fit(train_images, train_labels, epochs=50, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/50
1500/1500 [==============================] - 2s 2ms/step - loss: 0.2330 - accuracy: 0.9314 - val_loss: 0.1365 - val_accuracy: 0.9583
Epoch 2/50
1500/1500 [==============================] - 2s 1ms/step - loss: 0.0920 - accuracy: 0.9719 - val_loss: 0.0932 - val_accuracy: 0.9719
Epoch 3/50
1500/1500 [==============================] - 2s 1ms/step - loss: 0.0586 - accuracy: 0.9820 - val_loss: 0.0824 - val_accuracy: 0.9737
Epoch 4/50
1500/1500 [==============================] - 2s 1ms/step - loss: 0.0415 - accuracy: 0.9870 - val_loss: 0.0841 - val_accuracy: 0.9748
Epoch 5/50
1500/1500 [==============================] - 2s 1ms/step - loss: 0.0288 - accuracy: 0.9914 - val_loss: 0.0874 - val_accuracy: 0.9761
Epoch 6/50
1500/1500 [==============================] - 2s 1ms/step - loss: 0.0220 - accuracy: 0.9928 - val_loss: 0.0807 - val_accuracy: 0.9778
Epoch 7/50
1500/1500 [==============================] - 2s 1ms/step - loss: 0.0170 - accuracy: 0.9943 - val_loss: 0.0818 - val_accuracy:

In [14]:
model: Sequential = tuner.hypermodel.build(best_hyperparams)

model.fit(train_images, train_labels, epochs=50, validation_split=0.2)

Epoch 1/50
1500/1500 [==============================] - 2s 2ms/step - loss: 0.2287 - accuracy: 0.9327 - val_loss: 0.1250 - val_accuracy: 0.9647
Epoch 2/50
1500/1500 [==============================] - 2s 1ms/step - loss: 0.0915 - accuracy: 0.9723 - val_loss: 0.0957 - val_accuracy: 0.9724
Epoch 3/50
1500/1500 [==============================] - 2s 1ms/step - loss: 0.0582 - accuracy: 0.9824 - val_loss: 0.0899 - val_accuracy: 0.9736
Epoch 4/50
1500/1500 [==============================] - 2s 1ms/step - loss: 0.0407 - accuracy: 0.9873 - val_loss: 0.0788 - val_accuracy: 0.9763
Epoch 5/50
1500/1500 [==============================] - 2s 1ms/step - loss: 0.0299 - accuracy: 0.9906 - val_loss: 0.0866 - val_accuracy: 0.9737
Epoch 6/50
1500/1500 [==============================] - 2s 1ms/step - loss: 0.0224 - accuracy: 0.9931 - val_loss: 0.0852 - val_accuracy: 0.9761
Epoch 7/50
1500/1500 [==============================] - 2s 1ms/step - loss: 0.0180 - accuracy: 0.9942 - val_loss: 0.1064 - val_accuracy: